# Домашняя работа №5


# Банникова Екатерина НПМбд-02-19

# Решение СЛАУ методом LU-разложения и методом LUP-разложения

## 1) Решение СЛАУ с применением LU-разложения

LU-разложение — представление матрицы A в виде произведения двух матриц, A = LU, где U — верхняя треугольная матрица, а L —  нижняя унитреугольная, т. е. треугольная матрица с диагональными элементами, равными единице
(существует также вариант метода, в котором требование унитреугольности накладывают на матрицу U, а не на L).
LU-разложение используется для решения систем линейных уравнений, обращения матриц и вычисления определителя.
Метод LU-разложения имеет ограничения: LU-разложение существует только в том случае, когда матрица A квадратная и невырожденная, причём все её ведущие (угловые) главные миноры отличны от нуля.
Две треугольные матрицы: нижнюю L и верхнюю U — можно хранить совместно в одной квадратной матрице LU, без хранения диагонали матрицы L, элементы которой всегда равны единице.

In [ ]:
Для эффективного выполнения операция над матрицами и векторами используем возможности библиотеки numpy.

In [4]:
import numpy as np

Выполнение LU-разложения матрицы удобно реализовать в виде функции, принимающей исходную квадратную матрицу и возвращающей квадратную матрице LU, в которой содержатся совместно матрица U и матрица L без единиц главной диагонали матрицы L:
ниже главной диагонали матрицы LU размещаются элементы матрицы L, а выше и на диагонали — элементы матрицы U.

In [5]:
def LU_decomposition(A):
    LU = np.array(A, float)
    for k in range(LU.shape[0]-1):
        LU[k+1:, k] /= LU[k, k]
        LU[k+1:, k+1:] -= np.transpose([LU[k+1:, k]]) * LU[k, k+1:]
    return LU

Но прежде чем осуществлять LU-разложение, нужно проверить, возможно ли оно:

In [6]:
def LU_decomposition_is_possible(A):
    m = A.shape[0]
    n = A.shape[1]
    if m != n:
        return False, "LU-разложение невозможно: матрица не квадратная"
    if np.linalg.det(A) == 0:
        return False, "LU-разложение невозможно: матрица вырождена"
    for i in range(1, m):
        if np.linalg.det(A[:i, :i]) == 0:
            return False, "LU-разложение невозможно: главный минор вырожден"
    return True, "LU-разложение существует"

СЛАУ Ax = b, представленная при помощи LU-разложения в виде системы LUx = b,
решается последовательностью прямой подстановки, дающей решение системы Ly = b,
и обратной подстановки, дающей решение системы Ux = y.
Функция solve_LU вычисляет таким образом решение на основе полученного LU-разложения (в виде матрицы LU):

In [7]:
def solve_LU(LU, b):
    y = np.array(b, float)
    for i in range(1, len(y)):
        y[i] = b[i] - np.dot(LU[i, :i], y[:i])
    x = np.array(y)
    for i in range(len(y)-1, -1, -1):
        x[i] = (y[i] - np.dot(LU[i, i+1:], x[i+1:])) / LU[i, i]
    return x

Следует предусмотреть также функцию, реализующую от начала до конца нахождение решения системы Ax = b:

In [8]:
def decompose_to_LU_and_solve(A, b):
    LU = LU_decomposition(A)
    return solve_LU(LU, b)

При решении системы используется объединённая матрица LU.
Добавим также функцию, выделяющую две треугольные матрицы L и U из матрицы LU. Их можно использовать, чтобы протестировать правильность выполнения LU-разложения проверкой равенства A = L * U.

In [9]:
def L_and_U(LU):
    L = np.array(LU, float)
    U = np.array(LU, float)
    for i in range(LU.shape[0]):
        L[i, i] = 1
        L[i, i+1:] = 0
        U[i, :i] = 0
    return L, U

Кроме того, чтобы протестировать правильность нахождения решения, после получения вектора x будем проверять равенство Ax = b.

Помимо этого, для дополнительной проверки решения воспользуемся библиотекой scipy, сравнивая полученное решение с решением, полученным при помощи этой библиотеки.

In [10]:
import scipy.linalg

Вспомогательная функция для сравнения двух векторов (проверка на совпадение):

In [11]:
def compare_solutions(x1, x2):
    if np.allclose(x1, x2):
        return "совпадают"
    else:
        return "НЕ СОВПАДАЮТ"

Пусть СЛАУ задана матрицей коэффициентов

In [12]:
A = np.array([
               [1, 2, 3], 
               [4, 5, 6], 
               [7, 8, 10]  ])


и вектором правой части

In [13]:
b = np.array([  3,
                8,
                5   ])

Проверям, существует ли LU-разложение матрицы A, находим это разложение и вычисляем решение СЛАУ — вектор x.
При этом проверяем правильность LU-разложения (A = L * U) и
то, что полученное решение действительно соответствует СЛАУ (A * x = b);
а также сверяем полученное решение с результатом функции solve из набора функций линейной алгебры библиотеки scipy.

In [14]:
possible, possible_or_not = LU_decomposition_is_possible(A) 
print(possible_or_not)

if possible:
    LU = LU_decomposition(A)
    L, U = L_and_U(LU)

    print("LU-разложение:")
    print("матрица L:\n", L)
    print("матрица U:\n", U)
    print("матрица LU:\n", LU)

    L_mult_U = np.dot(L, U)

    print("Проверка LU-разложения:")
    print("произведение матриц L * U:\n", L_mult_U)
    arreq = np.allclose(L_mult_U, A)
    print("L*U = A, разложение выполнено правильно" if arreq else "Разложение неверное")

    print("Исходный столбец b:\n", b)

    x = solve_LU(LU, b)
    print("Решение СЛАУ x:\n", x)

    print("Проверка решения:")
    Ax = np.dot(A, x)
    print("произведение A * x:\n", Ax)
    arreq = np.allclose(Ax, b)
    print("A*x = b, получено правильное решение" if arreq else "Решение неверное")
    
    x_scipy = scipy.linalg.solve(A, b)
    print("Результат функции scipy.linalg.solve:", x_scipy)
    print("Полученное решение и результат функции scipy.linalg.solve", compare_solutions(x, x_scipy))

LU-разложение существует
LU-разложение:
матрица L:
 [[1. 0. 0.]
 [4. 1. 0.]
 [7. 2. 1.]]
матрица U:
 [[ 1.  2.  3.]
 [ 0. -3. -6.]
 [ 0.  0.  1.]]
матрица LU:
 [[ 1.  2.  3.]
 [ 4. -3. -6.]
 [ 7.  2.  1.]]
Проверка LU-разложения:
произведение матриц L * U:
 [[ 1.  2.  3.]
 [ 4.  5.  6.]
 [ 7.  8. 10.]]
L*U = A, разложение выполнено правильно
Исходный столбец b:
 [3 8 5]
Решение СЛАУ x:
 [-7.66666667 17.33333333 -8.        ]
Проверка решения:
произведение A * x:
 [3. 8. 5.]
A*x = b, получено правильное решение
Результат функции scipy.linalg.solve: [-7.66666667 17.33333333 -8.        ]
Полученное решение и результат функции scipy.linalg.solve совпадают


Допустим, теперь требуется решить СЛАУ с той же самой матрицей коэффициентов, но с другим вектором правой части:

In [15]:
bII = np.array([4, -3, 2])

В этом случае решение может быть быстро получено с применением уже полученного LU-разложения:

In [16]:
xII = solve_LU(LU, bII)
print("Решение системы с той же матрицей А, но с другой правой частью bII =", bII, ",\n    равно", xII)
print("Проверка решения:")
AxII = np.dot(A, xII)
print("Произведение A * xII:\n", AxII)
arreq = np.allclose(AxII, bII)
print("A*xII = bII, получено правильное решение" if arreq else "Решение неверное")
x_scipy = scipy.linalg.solve(A, bII)
print("Результат функции scipy.linalg.solve:", x_scipy)
print("Полученное решение и результат функции scipy.linalg.solve", compare_solutions(xII, x_scipy))

Решение системы с той же матрицей А, но с другой правой частью bII = [ 4 -3  2] ,
    равно [  3.33333333 -17.66666667  12.        ]
Проверка решения:
Произведение A * xII:
 [ 4. -3.  2.]
A*xII = bII, получено правильное решение
Результат функции scipy.linalg.solve: [  3.33333333 -17.66666667  12.        ]
Полученное решение и результат функции scipy.linalg.solve совпадают


Решение другой системы (с другой матрицей и другим вектором):

In [17]:
A2 = np.array([[6.3, -4, 3.7, 8], [9.1, 7, 0, 2.2], [4.75, -3, 5.66, 6], [6, 5, -6.18, 2]])

print("Исходная матрица A2:\n", A2)

possible, possible_or_not = LU_decomposition_is_possible(A2) 
print(possible_or_not)

if possible:
    b2 = np.array([2, 6, 3, 8])
    x2 = decompose_to_LU_and_solve(A2, b2)
    Ax2 = np.dot(A2, x2)
    arreq = np.allclose(Ax2, b2)
    print("Исходный столбец b2:\n", b2)
    print("Решение СЛАУ x2:\n", x2)
    print("Произведение A2 * x2:\n", Ax2)
    print("A2*x2 = b2, получено правильное решение" if arreq else "Решение неверное")
    x_scipy = scipy.linalg.solve(A2, b2)
    print("Результат функции scipy.linalg.solve:", x_scipy)
    print("Полученное решение и результат функции scipy.linalg.solve", compare_solutions(x2, x_scipy))


Исходная матрица A2:
 [[ 6.3  -4.    3.7   8.  ]
 [ 9.1   7.    0.    2.2 ]
 [ 4.75 -3.    5.66  6.  ]
 [ 6.    5.   -6.18  2.  ]]
LU-разложение существует
Исходный столбец b2:
 [2 6 3 8]
Решение СЛАУ x2:
 [-6.92183667  7.04156692  0.57991193  8.95352057]
Произведение A2 * x2:
 [2. 6. 3. 8.]
A2*x2 = b2, получено правильное решение
Результат функции scipy.linalg.solve: [-6.92183667  7.04156692  0.57991193  8.95352057]
Полученное решение и результат функции scipy.linalg.solve совпадают


## 2) Решение СЛАУ с применением LUP-разложения

Ограничения, присущие методу LU-разложения, удаётся преодолеть, применяя перестановку строк матрицы A (и соответствующую перестановку элементов вектора b) таким образом, чтобы LU-разложение существовало.
Этот подход реализуется методом LUP-разложения.
LUP-разложение матрицы A — это представление матрицы A в виде произведения P * A = L * U,
где L — нижняя унитреугольная матрица, U - верхняя треугольная матрица, P — матрица перестановок (матрица перестановок — это матрица, полученная из единичной матрицы перестановкой некоторых строк или столбцов).

Матрица перестановок P строится (из единичной матрицы) в процессе Гауссового исключения с перестановкой строк таким образом, чтобы при приведении матрицы A к ступенчатому виду на каждом j-м шаге — вверху оставшейся для обработки части j-го столбца (т.е. в элементе в j-й строке и j-м столбце) оказывался максимальный элемент среди элементов этого столбца в строках, начиная с j-й и ниже. При перестановке строк в матрице A переставляются соответственно и строки в матрице P.
Получение матрицы перестановок реализовано в виде следующей функции:

In [18]:
def pivot_matrix(A):
    M = np.array(A, float)
    m = M.shape[0]
    P = np.eye(m)
    for j in range(m):
        c = np.abs(M[j:, j])
        row = np.argmax(c) + j
        if j != row:
            P[[j, row], :] = P[[row, j], :]
            M[[j, row], :] = M[[row, j], :]
        if M[j, j] != 0:
            for r in range(j + 1, m):
                M[r, :] -= M[j, :] * M[r, j] / M[j, j]
    return P

В случае когда матрица коэффициентов квадратная, получив матрицу перестановок P, получить матрицу LU можно, воспользовавшись функцией LU-разложения и передав ей произведение матриц P и A:

In [19]:
def LUP_decomposition(A):
    P = pivot_matrix(A)
    PA = np.dot(P, A)
    LU = LU_decomposition(PA)
    return LU, P

Соответственно, вот функция для решения системы уравнений, исходя из полученного LUP-разложения
(внутри неё используется функция solve_LU, которой передаются матрица LU и произведение P * b):

In [20]:
def solve_LU_P(LU, P, b):
    Pb = np.dot(P, b)
    return solve_LU(LU, Pb)

Функция для решения системы уравнений:

In [21]:
def decompose_to_LUP_and_solve(A, b):
    LU, P = LUP_decomposition(A)
    return solve_LU_P(LU, P, b)

Функции для того, чтобы было удобнее сверять полученное LUP-разложение и решение с результатами функций библиотеки scipy:

In [22]:
def scipy_linalg_results(A, b):
    LU, P1 = scipy.linalg.lu_factor(A)
    P, L, U = scipy.linalg.lu(A)
    x = scipy.linalg.solve(A, b)
    return np.transpose(P), L, U, LU, x

def compare_results(P1, L1, U1, LU1, x1, P2, L2, U2, LU2, x2):
    if np.allclose(P1, P2) and \
       np.allclose(L1, L2) and \
       np.allclose(U1, U2) and \
       np.allclose(LU11, LU2) and \
       np.allclose(x1, x2):
          return "совпадают"
    else:
          return "НЕ СОВПАДАЮТ"

Здесь учтена особенность функции scipy.linalg.lu:
https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lu.html

  Examples ...
  
. >>> p, l, u = lu(A)

. >>> np.allclose(A - p @ l @ u, np.zeros((4, 4)))

  True
  
Видим, что scipy.linalg.lu выдаёт матрицу P, соответствующую равенству A = P * L * U.
Соответственно, чтобы получить матрицу P, соответствующую нашему равенству P * A = L * U,
матрицу P из результатов scipy.linalg.lu нужно транспонировать, что и делаем.

Применяем метод LUP-раздожения: находим матрицу перестановок и соответствующее LU-разложение, проверяем, что разложение выполнено верно; используя полученное разложение, находим решение СЛАУ и проверяем его. Далее находим решение второй СЛАУ.

In [23]:
print("Исходная матрица A:\n", A)

LU, P = LUP_decomposition(A)
L, U = L_and_U(LU)

print("LUP-разложение:")
print("матрица P:\n", P)
print("матрица L:\n", L)
print("матрица U:\n", U)
print("матрица LU:\n", LU)

P_mult_A = np.dot(P, A)
L_mult_U = np.dot(L, U)

print("Проверка LUP-разложения:")
print("произведение матриц P * A:\n", P_mult_A)
print("произведение матриц L * U:\n", L_mult_U)
arreq = np.allclose(L_mult_U, P_mult_A )
print("L*U = P*A, разложение выполнено правильно" if arreq else "Разложение неверное")

print("Исходный столбец b:\n", b)

x = solve_LU_P(LU, P, b)
print("Решение СЛАУ x:\n", x)

print("Проверка решения:")
Ax = np.dot(A, x)
print("произведение A * x:\n", Ax)
arreq = np.allclose(Ax, b)
print("A*x = b, получено правильное решение" if arreq else "Решение неверное")

P_scipy, L_scipy, U_scipy, LU_scipy, x_scipy = scipy_linalg_results(A, b)
print("Результаты LUP-разложения матрицы и решения системы функциями библиотеки scipy.linalg:")
print(" P   (scipy.linalg.lu):\n", P_scipy)
print(" L   (scipy.linalg.lu):\n", L_scipy)
print(" U   (scipy.linalg.lu):\n", U_scipy)
print(" LU   (scipy.linalg.lu_factor):\n", LU_scipy)
print(" x   (scipy.linalg.solve): ", x_scipy)
print("Полученное решение и результат функции scipy.linalg.solve", compare_solutions(x, x_scipy))

Исходная матрица A:
 [[ 1  2  3]
 [ 4  5  6]
 [ 7  8 10]]
LUP-разложение:
матрица P:
 [[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]
матрица L:
 [[1.         0.         0.        ]
 [0.14285714 1.         0.        ]
 [0.57142857 0.5        1.        ]]
матрица U:
 [[ 7.          8.         10.        ]
 [ 0.          0.85714286  1.57142857]
 [ 0.          0.         -0.5       ]]
матрица LU:
 [[ 7.          8.         10.        ]
 [ 0.14285714  0.85714286  1.57142857]
 [ 0.57142857  0.5        -0.5       ]]
Проверка LUP-разложения:
произведение матриц P * A:
 [[ 7.  8. 10.]
 [ 1.  2.  3.]
 [ 4.  5.  6.]]
произведение матриц L * U:
 [[ 7.  8. 10.]
 [ 1.  2.  3.]
 [ 4.  5.  6.]]
L*U = P*A, разложение выполнено правильно
Исходный столбец b:
 [3 8 5]
Решение СЛАУ x:
 [-7.66666667 17.33333333 -8.        ]
Проверка решения:
произведение A * x:
 [3. 8. 5.]
A*x = b, получено правильное решение
Результаты LUP-разложения матрицы и решения системы функциями библиотеки scipy.linalg:
 P   (scipy.linalg.lu)

In [24]:
print("Исходная матрица A2:\n", A2)
print("Исходный столбец b2:\n", b2)
x2 = decompose_to_LUP_and_solve(A2, b2)
Ax2 = np.dot(A2, x2)
arreq = np.allclose(Ax2, b2)
print("Решение СЛАУ x2:\n", x2)
print("Произведение A2 * x2:\n", Ax2)
print("A2*x2 = b2, получено правильное решение" if arreq else "Решение неверное")
x_scipy = scipy.linalg.solve(A2, b2)
print("Результат функции scipy.linalg.solve:", x_scipy)
print("Полученное решение и результат функции scipy.linalg.solve", compare_solutions(x2, x_scipy))

Исходная матрица A2:
 [[ 6.3  -4.    3.7   8.  ]
 [ 9.1   7.    0.    2.2 ]
 [ 4.75 -3.    5.66  6.  ]
 [ 6.    5.   -6.18  2.  ]]
Исходный столбец b2:
 [2 6 3 8]
Решение СЛАУ x2:
 [-6.92183667  7.04156692  0.57991193  8.95352057]
Произведение A2 * x2:
 [2. 6. 3. 8.]
A2*x2 = b2, получено правильное решение
Результат функции scipy.linalg.solve: [-6.92183667  7.04156692  0.57991193  8.95352057]
Полученное решение и результат функции scipy.linalg.solve совпадают


Проверим возможность применения методов LU-разложения и LUP-разложения к следующей матрице:

In [26]:
A3 = np.array([[1, -3, 7], [0, 0, 2], [2, 5, -1]])
print("Исходная матрица A3:\n", A3)

Исходная матрица A3:
 [[ 1 -3  7]
 [ 0  0  2]
 [ 2  5 -1]]


In [27]:
possible, possible_or_not = LU_decomposition_is_possible(A3) 
print(possible_or_not)

LU-разложение невозможно: главный минор вырожден


Получить LU-разложение для этой матрицы невозможно.

Получаем LUP-разложение этой матрицы и решаем СЛАУ:

In [28]:
LU, P = LUP_decomposition(A3)
L, U = L_and_U(LU)

print("LUP-разложение:")
print("матрица P:\n", P)
print("матрица L:\n", L)
print("матрица U:\n", U)
print("матрица LU:\n", LU)

P_mult_A = np.dot(P, A3)
L_mult_U = np.dot(L, U)

print("Проверка LUP-разложения:")
print("произведение матриц P * A3:\n", P_mult_A)
print("произведение матриц L * U:\n", L_mult_U)
arreq = np.allclose(L_mult_U, P_mult_A )
print("L*U = P*A3, разложение выполнено правильно" if arreq else "Разложение неверное")

print("Исходный столбец b:\n", b)

x3 = solve_LU_P(LU, P, b)
print("Решение СЛАУ x3:\n", x3)

print("Проверка решения:")
Ax3 = np.dot(A3, x3)
print("произведение A3 * x3:\n", Ax)
arreq = np.allclose(Ax3, b)
print("A3*x3 = b, получено правильное решение" if arreq else "Решение неверное")

P_scipy, L_scipy, U_scipy, LU_scipy, x_scipy = scipy_linalg_results(A3, b)
print("Результаты LUP-разложения матрицы и решения системы функциями библиотеки scipy.linalg:")
print(" P   (scipy.linalg.lu):\n", P_scipy)
print(" L   (scipy.linalg.lu):\n", L_scipy)
print(" U   (scipy.linalg.lu):\n", U_scipy)
print(" LU   (scipy.linalg.lu_factor):\n", LU_scipy)
print(" x   (scipy.linalg.solve): ", x_scipy)
print("Полученное решение и результат функции scipy.linalg.solve", compare_solutions(x3, x_scipy))

LUP-разложение:
матрица P:
 [[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]
матрица L:
 [[ 1.   0.   0. ]
 [ 0.5  1.   0. ]
 [ 0.  -0.   1. ]]
матрица U:
 [[ 2.   5.  -1. ]
 [ 0.  -5.5  7.5]
 [ 0.   0.   2. ]]
матрица LU:
 [[ 2.   5.  -1. ]
 [ 0.5 -5.5  7.5]
 [ 0.  -0.   2. ]]
Проверка LUP-разложения:
произведение матриц P * A3:
 [[ 2.  5. -1.]
 [ 1. -3.  7.]
 [ 0.  0.  2.]]
произведение матриц L * U:
 [[ 2.  5. -1.]
 [ 1. -3.  7.]
 [ 0.  0.  2.]]
L*U = P*A3, разложение выполнено правильно
Исходный столбец b:
 [3 8 5]
Решение СЛАУ x3:
 [-8.90909091  5.36363636  4.        ]
Проверка решения:
произведение A3 * x3:
 [3. 8. 5.]
A3*x3 = b, получено правильное решение
Результаты LUP-разложения матрицы и решения системы функциями библиотеки scipy.linalg:
 P   (scipy.linalg.lu):
 [[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]
 L   (scipy.linalg.lu):
 [[ 1.   0.   0. ]
 [ 0.5  1.   0. ]
 [ 0.  -0.   1. ]]
 U   (scipy.linalg.lu):
 [[ 2.   5.  -1. ]
 [ 0.  -5.5  7.5]
 [ 0.   0.   2. ]]
 LU   (scipy.linalg.lu_factor):

Решим СЛАУ со случайной матрицей 3x3.
Решение методом LU-разложения получаем в том случае, если LU-разложение существует.
Решение методом LU-разложения получаем в любом случае.

In [31]:
A4 = np.array([np.random.randint(-10, 10, 3),
               np.random.randint(-10, 10, 3),
               np.random.randint(-10, 10, 3)])
b4 = np.array(np.random.randint(-10, 10, 3))
print("Исходная матрица A4:\n", A4)
print("Исходный столбец b:\n", b4)
possible, possible_or_not = LU_decomposition_is_possible(A4) 
print(possible_or_not)
if possible:
    x41 = decompose_to_LU_and_solve(A4, b4)
    Ax4 = np.dot(A4, x41)
    arreq = np.allclose(Ax4, b4)
    print("Решение СЛАУ с помощью LU-разложения x41:\n", x41)
    print("Произведение A4 * x41:\n", Ax4)
    print("A4*x41 = b4, получено правильное решение" if arreq else "Решение неверное")
    x_scipy = scipy.linalg.solve(A4, b4)
    print("Результат функции scipy.linalg.solve:", x_scipy)
    print("Полученное решение и результат функции scipy.linalg.solve", compare_solutions(x41, x_scipy))
x4 = decompose_to_LUP_and_solve(A4, b4)
Ax4 = np.dot(A4, x4)
arreq = np.allclose(Ax4, b4)
print("Решение СЛАУ с помощью LUP-разложения x4:\n", x4)
print("Произведение A4 * x4:\n", Ax4)
print("A4*x4 = b4, получено правильное решение" if arreq else "Решение неверное")
x_scipy = scipy.linalg.solve(A4, b4)
print("Результат функции scipy.linalg.solve:", x_scipy)
print("Полученное решение и результат функции scipy.linalg.solve", compare_solutions(x4, x_scipy))

Исходная матрица A4:
 [[-3  0  0]
 [-6  0 -3]
 [-5 -8 -8]]
Исходный столбец b:
 [ 9  7 -9]
LU-разложение невозможно: главный минор вырожден
Решение СЛАУ с помощью LUP-разложения x4:
 [-3.         -0.66666667  3.66666667]
Произведение A4 * x4:
 [ 9.  7. -9.]
A4*x4 = b4, получено правильное решение
Результат функции scipy.linalg.solve: [-3.         -0.66666667  3.66666667]
Полученное решение и результат функции scipy.linalg.solve совпадают


# Вывод:

Применительно к задаче решения СЛАУ — LU-разложение и LUP-разложение дают особый выигрыш, когда нужно находить решение нескольких СЛАУ с одинаковой матрицей коэффициентов, но различающихся правой частью.
Не всегда возможно произвести LU-разложение матрицы. Но LUP-разложение всегда возможно. LU-разложение подходит только для обратимой матрицы A с невырожденными угловыми главными минорами.
Существование LU-разложения можно проверить, но, по видимости, на практике стоит использовать метод LU-разложение в случаях, если заведомо известно, что для данных матриц оно существует. В общем случае целесообразно применять метод LUP-разложения.